In [79]:
from typing import Any, Dict, List, Optional, cast
from typing_extensions import TypedDict
import pystac
from pystac.extensions.eo import EOExtension
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.view import ViewExtension
import csv
from datetime import datetime
import gzip
from io import StringIO
from urllib.request import urlopen

# Here we use a TypedDict to define the structure of the Landsat scene dictionaries
# we will be working with. You can read more about the TypedDict class here:
# https://docs.python.org/3/library/typing.html#typing.TypedDict
class Scene(TypedDict):
    productId: str
    entityId: str
    acquisitionDate: str
    cloudCover: str
    processingLevel: str
    path: str
    row: str
    min_lat: str
    min_lon: str
    max_lat: str
    max_lon: str
    download_url: str

# Collection 1 data
url = 'https://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz'

# Read and unzip the content
response = urlopen(url)
gunzip_response = gzip.GzipFile(fileobj=response)
content = gunzip_response.read()

# Read the scenes in as dictionaries
scenes: List[Scene] = list(map(lambda s: cast(Scene, s), csv.DictReader(StringIO(content.decode("utf-8")))))


In [ ]:
len(scenes)
scenes[0]

lat, lon = 38.6383, -120.2577
location_name = "target"


def keep_scene(scene: Scene) -> bool:
    contains_location = float(scene['min_lat']) < lat and float(scene['max_lat']) > lat and \
                        float(scene['min_lon']) < lon and float(scene['max_lon']) > lon
    return contains_location
        
location_scenes = [scene for scene in scenes if keep_scene(scene)]
len(location_scenes)

scene = location_scenes[0]
scene
def get_asset_url(scene: Scene, suffix: str) -> str:
    product_id = scene['productId']
    download_url = scene['download_url']    
    asset_filename = '{}_{}'.format(product_id, suffix)
    return download_url.replace('index.html', asset_filename)

stac_io = pystac.StacIO.default()
mtl_url = get_asset_url(scene, 'MTL.txt')
print(stac_io.read_text(mtl_url))

def get_metadata(url: str) -> Dict[str, Any]:
    """ Convert Landsat MTL file to dictionary of metadata values """
    mtl = {}
    mtl_text = stac_io.read_text(url)
    for line in mtl_text.split('\n'):
        meta = line.replace('\"', "").strip().split('=')
        if len(meta) > 1:
            key = meta[0].strip()
            item = meta[1].strip()
            if key != "GROUP" and key != "END_GROUP":
                mtl[key] = item
    return mtl
metadata = get_metadata(mtl_url)
metadata


In [41]:
def get_item_id(metadata: Dict[str, Any]) -> str:
    return cast(str, metadata['LANDSAT_SCENE_ID'][:-5])


item_id = get_item_id(metadata)
item_id

from dateutil.parser import parse

def get_datetime(metadata: Dict[str, Any]) -> datetime:
    return parse('%sT%s' % (metadata['DATE_ACQUIRED'], metadata['SCENE_CENTER_TIME']))

item_datetime = get_datetime(metadata)
item_datetime

def get_bbox(metadata: Dict[str, Any]) -> List[float]:
    coords = [[
        [float(metadata['CORNER_UL_LON_PRODUCT']), float(metadata['CORNER_UL_LAT_PRODUCT'])],
        [float(metadata['CORNER_UR_LON_PRODUCT']), float(metadata['CORNER_UR_LAT_PRODUCT'])],
        [float(metadata['CORNER_LR_LON_PRODUCT']), float(metadata['CORNER_LR_LAT_PRODUCT'])],
        [float(metadata['CORNER_LL_LON_PRODUCT']), float(metadata['CORNER_LL_LAT_PRODUCT'])],
        [float(metadata['CORNER_UL_LON_PRODUCT']), float(metadata['CORNER_UL_LAT_PRODUCT'])]
    ]]
    lats = [c[1] for c in coords[0]]
    lons = [c[0] for c in coords[0]]
    return [min(lons), min(lats), max(lons), max(lats)]

item_bbox = get_bbox(metadata)
item_bbox



def get_geometry(scene: Scene, bbox: List[float]) -> Dict[str, Any]:
    url = get_asset_url(scene, 'ANG.txt')
    sz = []
    coords = []
    ang_text = stac_io.read_text(url)
    for line in ang_text.split('\n'):
        if 'BAND01_NUM_L1T_LINES' in line or 'BAND01_NUM_L1T_SAMPS' in line:
            sz.append(float(line.split('=')[1]))
        if 'BAND01_L1T_IMAGE_CORNER_LINES' in line or 'BAND01_L1T_IMAGE_CORNER_SAMPS' in line:
            coords.append([float(l) for l in line.split('=')[1].strip().strip('()').split(',')])
        if len(coords) == 2:
            break
    dlon = bbox[2] - bbox[0]
    dlat = bbox[3] - bbox[1]
    lons = [c/sz[1] * dlon + bbox[0] for c in coords[1]]
    lats = [((sz[0] - c)/sz[0]) * dlat + bbox[1] for c in coords[0]]
    coordinates = [[
        [lons[0], lats[0]], [lons[1], lats[1]], [lons[2], lats[2]], [lons[3], lats[3]], [lons[0], lats[0]]
    ]]
    
    return {'type': 'Polygon', 'coordinates': coordinates}

item_geometry = get_geometry(scene, item_bbox)
item_geometry



import json

print(json.dumps(item_geometry, indent=2))


item = pystac.Item(id=item_id, 
                  datetime=item_datetime,
                  geometry=item_geometry,
                  bbox=item_bbox,
                  properties={})

item.common_metadata.gsd = 30.0

eo_ext = EOExtension.ext(item, add_if_missing=True)


def get_cloud_cover(metadata: Dict[str, Any]) -> float:
    return float(metadata['CLOUD_COVER'])

eo_ext.cloud_cover = get_cloud_cover(metadata)
eo_ext.cloud_cover


from pystac.extensions.eo import Band

class BandInfo(TypedDict):
    band: Band
    gsd: float


landsat_band_info: Dict[str, BandInfo] = {
    'B1': {
        'band': Band.create(name="B1", common_name="coastal", center_wavelength=0.48, full_width_half_max=0.02),
        'gsd': 30.0
    },
    'B2': {
        'band': Band.create(name="B2", common_name="blue", center_wavelength=0.44, full_width_half_max=0.06),
        'gsd': 30.0
    },
    'B3': {
        'band': Band.create(name="B3", common_name="green", center_wavelength=0.56, full_width_half_max=0.06),
        'gsd': 30.0
    },
    'B4': {
        'band': Band.create(name="B4", common_name="red", center_wavelength=0.65, full_width_half_max=0.04),
        'gsd': 30.0
    },
    'B5': {
        'band': Band.create(name="B5", common_name="nir", center_wavelength=0.86, full_width_half_max=0.03),
        'gsd': 30.0
    },
    'B6': {
        'band': Band.create(name="B6", common_name="swir16", center_wavelength=1.6, full_width_half_max=0.08),
        'gsd': 30.0
    },
    'B7': {
        'band': Band.create(name="B7", common_name="swir22", center_wavelength=2.2, full_width_half_max=0.2),
        'gsd': 30.0
    },
    'B8': {
        'band': Band.create(name="B8", common_name="pan", center_wavelength=0.59, full_width_half_max=0.18),
        'gsd': 15.0
    },
    'B9': {
        'band': Band.create(name="B9", common_name="cirrus", center_wavelength=1.37, full_width_half_max=0.02),
        'gsd': 30.0
    },
    'B10': {
        'band': Band.create(name="B10", common_name="lwir11", center_wavelength=10.9, full_width_half_max=0.8),
        'gsd': 100.0
    },
    'B11': {
        'band': Band.create(name="B11", common_name="lwir12", center_wavelength=12.0, full_width_half_max=1.0),
        'gsd': 100.0
    }
}


def get_other_assets(scene: Scene) -> Dict[str, Dict[str, Any]]:
    return {
    'thumbnail': {
        'href': get_asset_url(scene, 'thumb_large.jpg'),
        'media_type': pystac.MediaType.JPEG
    },
    'index': {
        'href': get_asset_url(scene, 'index.html'),
        'media_type': 'application/html'
    },
    'ANG': {
        'href': get_asset_url(scene, 'ANG.txt'),
        'media_type': 'text/plain'
    },
    'MTL': {
        'href': get_asset_url(scene, 'MTL.txt'),
        'media_type': 'text/plain'
    },
    'BQA': {
        'href': get_asset_url(scene, 'BQA.TIF'),
        'media_type': pystac.MediaType.GEOTIFF
    }
}



{
  "type": "Polygon",
  "coordinates": [
    [
      [
        -121.0071546594264,
        39.96398646982859
      ],
      [
        -118.7815247241364,
        39.60864128953914
      ],
      [
        -119.25726848858399,
        37.82191379063866
      ],
      [
        -121.4791487579099,
        38.18741109630396
      ],
      [
        -121.0071546594264,
        39.96398646982859
      ]
    ]
  ]
}


In [42]:
def add_assets(item: pystac.Item, scene: Scene) -> None:
    # Add bands
    for band_id, band_info in landsat_band_info.items():
        band_url = get_asset_url(scene, '{}.TIF'.format(band_id))
        asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
        bands = [band_info['band']]
        asset_eo_ext = EOExtension.ext(asset)
        asset_eo_ext.bands = bands
        item.add_asset(band_id, asset)
        
        # If this asset has a different GSD than the item, set it on the asset
        if band_info['gsd'] != item.common_metadata.gsd:
            asset.common_metadata.gsd = band_info["gsd"]
        
    # Add other assets
    for asset_id, asset_info in get_other_assets(scene).items():
        item.add_asset(asset_id, 
                       pystac.Asset(href=asset_info['href'], media_type=asset_info['media_type']))
    
    
add_assets(item, scene)

item.assets['B10'].to_dict()
def get_epsg(metadata: Dict[str, Any], min_lat: float, max_lat: float) -> Optional[int]:
    if 'UTM_ZONE' in metadata:
        center_lat = (min_lat + max_lat)/2.0
        return int(('326' if center_lat > 0 else '327') + metadata['UTM_ZONE'])
    else:
        return None
    
proj_ext = ProjectionExtension.ext(item, add_if_missing=True)
assert item.bbox is not None
proj_ext.epsg = get_epsg(metadata, item.bbox[1], item.bbox[3]) 
proj_ext.epsg   

def get_view_info(metadata: Dict[str, Any]) -> Dict[str, float]:
    return { 'sun_azimuth': float(metadata['SUN_AZIMUTH']),
             'sun_elevation': float(metadata['SUN_ELEVATION']) }

view_ext = ViewExtension.ext(item, add_if_missing=True)
view_info = get_view_info(metadata)
view_ext.sun_azimuth = view_info['sun_azimuth']
view_ext.sun_elevation = view_info['sun_elevation']
item.properties

location_name = 'target'
collection_id = '{}-landsat-8'.format(location_name)
collection_id


collection_title = 'Landsat 8 images over {}'.format(location_name)
collection_title

collection_description = '''### {} Landsat 8

A collection of Landsat 8 scenes around {} in 2020.
'''.format(location_name, location_name)
print(collection_description)

AttributeError: 'dict' object has no attribute 'add_asset'

In [ ]:
from datetime import datetime

spatial_extent = pystac.SpatialExtent([[-180., -90., 180., 90.]])
collection_extent = pystac.Extent(spatial_extent, temporal_extent)

collection = pystac.Collection(id=collection_id,
                     title=collection_title,
                     description=collection_description,
                     extent=collection_extent)

In [ ]:
collection_license = 'PDDL-1.0'
collection.providers = [
    pystac.Provider(name='USGS', roles=[pystac.ProviderRole.PRODUCER], url='https://landsat.usgs.gov/'),
    pystac.Provider(name='Planet Labs', roles=[pystac.ProviderRole.PROCESSOR], url='https://github.com/landsat-pds/landsat_ingestor'),
    pystac.Provider(name='AWS', roles=[pystac.ProviderRole.HOST], url='https://landsatonaws.com/')
]


def item_from_scene(scene: Scene) -> pystac.Item:
    mtl_url = get_asset_url(scene, 'MTL.txt')
    metadata = get_metadata(mtl_url)
    
    bbox = get_bbox(metadata)
    item = pystac.Item(id=get_item_id(metadata), 
                       datetime=get_datetime(metadata),
                       geometry=get_geometry(scene, bbox),
                       bbox=bbox,
                       properties={})
    
    item.common_metadata.gsd = 30.0
    
    item_eo_ext = EOExtension.ext(item, add_if_missing=True)
    item_eo_ext.cloud_cover = get_cloud_cover(metadata)
    
    add_assets(item, scene)
    
    item_proj_ext = ProjectionExtension.ext(item, add_if_missing=True)
    assert item.bbox is not None
    item_proj_ext.epsg = get_epsg(metadata, item.bbox[1], item.bbox[3]) 

    item_view_ext = ViewExtension.ext(item, add_if_missing=True)
    view_info = get_view_info(metadata)
    item_view_ext.sun_azimuth = view_info['sun_azimuth']
    item_view_ext.sun_elevation = view_info['sun_elevation']

    #item.validate()
    
    return item

item_list = []
for scene in location_scenes:
    item = item_from_scene(scene)
    item_list.append(item.to_dict())




In [50]:
import pystac
import shapley

bbox = [ -120.2577, 38.6383, -120.2975, 38.6684]
polygon = shapely.geometry.box(bbox[0], bbox[1], bbox[2], bbox[3], ccw=True)
search = search(collections=collection, intersects=polygon1, datetime=date_range)



ImportError: cannot import name 'geometry' from 'shapley' (/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/shapley/__init__.py)

In [62]:
item_list = []
for scene in location_scenes:
    item = item_from_scene(scene)
    item_list.append(item.to_dict())

    


In [63]:
item_list[0]

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'LC80430332017094',
 'properties': {'gsd': 30.0,
  'eo:cloud_cover': 35.88,
  'proj:epsg': 32610,
  'view:sun_azimuth': 144.49466908,
  'view:sun_elevation': 51.95059498,
  'datetime': '2017-04-04T18:39:03.017934Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-121.0071546594264, 39.96398646982859],
    [-118.7815247241364, 39.60864128953914],
    [-119.25726848858399, 37.82191379063866],
    [-121.4791487579099, 38.18741109630396],
    [-121.0071546594264, 39.96398646982859]]]},
 'links': [],
 'assets': {'B1': {'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/043/033/LC08_L1TP_043033_20170404_20170414_01_T1/LC08_L1TP_043033_20170404_20170414_01_T1_B1.TIF',
   'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>,
   'eo:bands': [{'name': 'B1',
     'common_name': 'coastal',
     'center_wavelength': 0.48,
     'full_width_half_max': 0.02}]},
  'B2': {'href': 'https://s3-us-west-2.a

In [ ]:

item = item_list[0]

item[str('B1')]


KeyError: 'B1'

In [75]:
item = item_list[0]['assets']


bands_intrest = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B6', 'B8', 'B9', 'B10', 'B11']

links = []
for band in bands_intrest:
    links.append(item[band]["href"])
item

#links

{'B1': {'href': 's3://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/043/033/LC08_L1TP_043033_20170404_20170414_01_T1/LC08_L1TP_043033_20170404_20170414_01_T1_B1.TIF',
  'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>,
  'eo:bands': [{'name': 'B1',
    'common_name': 'coastal',
    'center_wavelength': 0.48,
    'full_width_half_max': 0.02}]},
 'B2': {'href': 's3://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/043/033/LC08_L1TP_043033_20170404_20170414_01_T1/LC08_L1TP_043033_20170404_20170414_01_T1_B2.TIF',
  'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>,
  'eo:bands': [{'name': 'B2',
    'common_name': 'blue',
    'center_wavelength': 0.44,
    'full_width_half_max': 0.06}]},
 'B3': {'href': 's3://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/043/033/LC08_L1TP_043033_20170404_20170414_01_T1/LC08_L1TP_043033_20170404_20170414_01_T1_B3.TIF',
  'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-opt

In [80]:
tmp = 's3://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/043/033/LC08_L1TP_043033_20170404_20170414_01_T1/LC08_L1TP_043033_20170404_20170414_01_T1_B3.TIF'
with rasterio.open(tmp) as geo_fp:
    tmp = geo_fp.read()

RasterioIOError: The AWS Access Key Id you provided does not exist in our records.

In [72]:
import rasterio